In [4]:
import pandas as pd
import numpy as np

In [ ]:
# 사이트 변경 이전 데이터 크롤링 방법
# url = "http://seoul.go.kr/coronaV/coronaStatus.do
# table = pd.read_html(url)

# read_html 은 사이트 내의 html 태그를 불러옴
# table[0].T

# table 변수 안에 있는 table 태그의 값을 하나씩 읽어보며 확진자 목록이 있는 테이블을 가져옴
# df = table[4]

# last_day = df.loc[0, "확진일"]
# last_day = last_day.replace(".", "_")

#file_name = f"seoul-covid19_{last_day}.scv"

#df.to_csv(file_name, index=False)
#엑셀로 저장시 encoding='cp949'

#pd.read_csv(file_name)

#### 데이터 수집 방법
1. 수집하고자 하는 페이지의 URL을 알아본다.
2. 파이썬의 작은 브라우저 requests를 통해 URL에 접근한다.
3. response.status_code가 200 OK 라면 정상 응답입니다.
4. request의 response값에서 response.txt만 받아옵니다.
5. html 텍스트를 받아왔다면 bs(reponse.text, 'html.parser')로 html을 해석합니다.
6. soup.select를 통해 원하는 태그에 접근합니다.
7. 목록을 먼저 받아옵니다.
8. 목록에서 행을 하나씩 가져옵니다.
9. 행을 모아서 데이터프레임으로 만듭니다.

#### robot.txt를 확인할 것
disallow: /일 경우 수집 금지

#### chrome 소스코드 보는 법
1. 검사
2. network > XHR or JS or Doc > URL
3. Preview에서 JSON 형식으로 볼 수 있음
ex) data = data_json["data"]
    pd.DataFrame(data)
    
### 서울시 코로나 발생현황 데이터 수집 순서
1. 페이지별 데이터를 수집
2. 전체 페이지를 수집
3. pd.concat([전체데이터리스트])로 떼이터를 하나로 병합
4. 데이터 전처리(html 태그 제거)
5. to_csv로 전체 데이터 병합
6. pd.read_csv로 데이터가 잘 저장되었는지 읽어와서 확인

In [2]:
url = "http://www.seoul.go.kr/coronaV/coronaStatus.do"
url

'http://www.seoul.go.kr/coronaV/coronaStatus.do'

In [5]:
table = pd.read_html(url)

In [6]:
len(table)

8

In [7]:
# read_html은 사이트 내의 html 태그를 불러옴
table[0]

,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,동대문구,동작구,마포구
0,6389,3365,2296,3966,4946,3078,3475,1790,3544,2311,2968,3817,3350
1,+45,+25,+13,+30,+29,+19,+65,+17,+23,+17,+24,+26,+47
2,서대문구,서초구,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구,기타
3,2423,3995,2548,3353,5460,2995,3969,2500,3925,1368,1392,3408,6452
4,+29,+24,+17,+20,+67,+17,+42,+7,+30,+12,+20,+8,+46


In [8]:
# table 변수 안에 있는 table 태그값을 하나씩 읽어보며 확진자 목록이 있는 테이블을 가져옴
table[1]

,강남구,강동구,강북구,강서구,관악구,광진구
0,6389,3365,2296,3966,4946,3078
1,+45,+25,+13,+30,+29,+19
2,구로구,금천구,노원구,도봉구,동대문구,동작구
3,3475,1790,3544,2311,2968,3817
4,+65,+17,+23,+17,+24,+26
5,마포구,서대문구,서초구,성동구,성북구,송파구
6,3350,2423,3995,2548,3353,5460
7,+47,+29,+24,+17,+20,+67
8,양천구,영등포구,용산구,은평구,종로구,중구
9,2995,3969,2500,3925,1368,1392


In [68]:
table[5]

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황


# 11월 중순 이후 데이터 공개방식 변경

In [14]:
import requests

In [19]:
#f-string
url = "https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax_pre.php?draw=1"
# url = f"{url}&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=desc"
url = f"{url}&start=0&length=100"
# url = f"{url}&search%5Bvalue%5D=&search%5Bregex%5D=true&_=1631758944138"
url

'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax_pre.php?draw=1&start=0&length=100'

In [24]:
# 소스코드가 get이므로 get 사용
response = requests.get(url)
data_json = response.json()
pd.DataFrame(data_json["data"])

,0,1,2,3,4,5,6
0,<p class='corona19_no'>40000</p>,128607,2021-05-11,서초구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
1,<p class='corona19_no'>39999</p>,128576,2021-05-11,양천구,-,양천구 소재 직장 관련('21.5.),<b class='status1'>퇴원</b>
2,<p class='corona19_no'>39998</p>,128347,2021-05-11,강북구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
3,<p class='corona19_no'>39997</p>,128724,2021-05-11,강남구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
4,<p class='corona19_no'>39996</p>,128478,2021-05-11,양천구,-,감염경로 조사중,<b class='status1'>퇴원</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>39905</p>,128645,2021-05-11,타시도,-,감염경로 조사중,<b class='status1'>퇴원</b>
96,<p class='corona19_no'>39904</p>,128539,2021-05-11,중랑구,-,타시도 확진자 접촉,<b class='status1'>퇴원</b>
97,<p class='corona19_no'>39903</p>,128508,2021-05-11,도봉구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
98,<p class='corona19_no'>39902</p>,128512,2021-05-11,도봉구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>


In [26]:
records_total = data_json['recordsTotal']
records_total

40000

# 함수로 만들기

In [29]:
def get_seoul_covid19_100(page_no):
    """
    page_no : 입력값으로 페이지 번호를 입력하면 해당 번호의 데이터를 가져옴
    start_no : 입력받은 page_no로
    """
    start_no = (page_no - 1) * 100
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax_pre.php?draw={page_no}"
    url = f"{url}&order%5B0%5D%5Bdir%5D=desc&start={start_no}&length=100"
    response = requests.get(url)
    data_json = response.json()
    return data_json

In [30]:
# 데이터가 있을 때
get_seoul_covid19_100(1)

{'draw': 1,
 'recordsTotal': 40000,
 'recordsFiltered': 40000,
 'data': [["<p class='corona19_no'>40000</p>",
   '128607',
   '2021-05-11',
   '서초구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39999</p>",
   '128576',
   '2021-05-11',
   '양천구',
   '-',
   "양천구 소재 직장 관련('21.5.)",
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39998</p>",
   '128347',
   '2021-05-11',
   '강북구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39997</p>",
   '128724',
   '2021-05-11',
   '강남구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39996</p>",
   '128478',
   '2021-05-11',
   '양천구',
   '-',
   '감염경로 조사중',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39995</p>",
   '128410',
   '2021-05-11',
   '노원구',
   '-',
   '감염경로 조사중',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39994</p>",
   '128572',
   '2021-05-11',
   '금천구',
   '-',
   '감염경로 조사중',
  

In [31]:
get_seoul_covid19_100(2)

{'draw': 2,
 'recordsTotal': 40000,
 'recordsFiltered': 40000,
 'data': [["<p class='corona19_no'>39900</p>",
   '128409',
   '2021-05-11',
   '관악구',
   '-',
   '감염경로 조사중',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39899</p>",
   '128443',
   '2021-05-11',
   '동작구',
   '-',
   '강남구 소재 학원 관련',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39898</p>",
   '128527',
   '2021-05-11',
   '중랑구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39897</p>",
   '128511',
   '2021-05-11',
   '도봉구',
   '-',
   '감염경로 조사중',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39896</p>",
   '128522',
   '2021-05-11',
   '중랑구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39895</p>",
   '128507',
   '2021-05-11',
   '강북구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39894</p>",
   '127848',
   '2021-05-10',
   '기타',
   '-',
   '노원구 직장/지인모임 관련',
   "<

In [32]:
# 데이터가 없을 때
get_seoul_covid19_100(1000)

{'draw': 1000, 'recordsTotal': 40000, 'recordsFiltered': 40000, 'data': []}

# 전체 데이터 가져오기

In [33]:
# time.sleep으로 시차를 두기 위해
import time
# tqdm: 진행상태를 표시하기 위해
from tqdm import trange

In [52]:
# 전체 페이지를 가져오기 전에 일부 페이지만 실행
page_list = []
# 데이터가 제대로 수집 되는지 앞 부분 3페이지 정도만 확인하고 전체 페이지를 수집
# 처음부터 전체 페이지를 수집하면 중간에 오류가 발생해도 찾기 어렵기 때문
all_page = 3
for page_no in trange(1, all_page + 1):
    one_page = get_seoul_covid19_100(page_no)
    if len(one_page["data"]) > 0:
        one_page = pd.DataFrame(one_page["data"])
        page_list.append(one_page)
# 서버에 부담을 주지 않기 위해 0.5초 텀
        time.sleep(0.5)
    else:
        break

100%|██████████| 3/3 [00:02<00:00,  1.20it/s]


In [53]:
# 가져온 데이터가 맞는지 슬라이싱으로 확인
# page_list[:2]
pd.concat(page_list)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>40000</p>,128607,2021-05-11,서초구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
1,<p class='corona19_no'>39999</p>,128576,2021-05-11,양천구,-,양천구 소재 직장 관련('21.5.),<b class='status1'>퇴원</b>
2,<p class='corona19_no'>39998</p>,128347,2021-05-11,강북구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
3,<p class='corona19_no'>39997</p>,128724,2021-05-11,강남구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
4,<p class='corona19_no'>39996</p>,128478,2021-05-11,양천구,-,감염경로 조사중,<b class='status1'>퇴원</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>39705</p>,127625,2021-05-09,강남구,-,강남구 소재 초등학교 관련,<b class='status1'>퇴원</b>
96,<p class='corona19_no'>39704</p>,127626,2021-05-09,강남구,-,강남구 소재 초등학교 관련,<b class='status1'>퇴원</b>
97,<p class='corona19_no'>39703</p>,127689,2021-05-09,광진구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
98,<p class='corona19_no'>39702</p>,127518,2021-05-09,노원구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>


In [58]:
def get_multi_page_list(start_page, end_page):
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        one_page = get_seoul_covid19_100(page_no)
        if len(one_page['data']) > 0:
            one_page = pd.DataFrame(one_page['data'])
            page_list.append(one_page)
            time.sleep(0.5)
        else:
            return page_list
    return page_list

In [61]:
# 데이터 확인
start_page = 1
end_page = 88
page_list = get_multi_page_list(start_page, end_page)
page_list[:1]

100%|██████████| 88/88 [01:39<00:00,  1.13s/it]


[                                   0       1           2    3  4  \
 0   <p class='corona19_no'>40000</p>  128607  2021-05-11  서초구  -   
 1   <p class='corona19_no'>39999</p>  128576  2021-05-11  양천구  -   
 2   <p class='corona19_no'>39998</p>  128347  2021-05-11  강북구  -   
 3   <p class='corona19_no'>39997</p>  128724  2021-05-11  강남구  -   
 4   <p class='corona19_no'>39996</p>  128478  2021-05-11  양천구  -   
 ..                               ...     ...         ...  ... ..   
 95  <p class='corona19_no'>39905</p>  128645  2021-05-11  타시도  -   
 96  <p class='corona19_no'>39904</p>  128539  2021-05-11  중랑구  -   
 97  <p class='corona19_no'>39903</p>  128508  2021-05-11  도봉구  -   
 98  <p class='corona19_no'>39902</p>  128512  2021-05-11  도봉구  -   
 99  <p class='corona19_no'>39901</p>  128331  2021-05-11  용산구  -   
 
                        5                          6  
 0              기타 확진자 접촉  <b class='status1'>퇴원</b>  
 1   양천구 소재 직장 관련('21.5.)  <b class='status1'>퇴원</b>  
 2   

In [63]:
# 하나의 데이터 프레임으로 만듦
df_all = pd.concat(page_list)
df_all.shape

(8800, 7)

In [69]:
cols = table[5].columns.tolist()

In [71]:
df_all.columns = cols

In [72]:
df_all.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,<p class='corona19_no'>40000</p>,128607,2021-05-11,서초구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
1,<p class='corona19_no'>39999</p>,128576,2021-05-11,양천구,-,양천구 소재 직장 관련('21.5.),<b class='status1'>퇴원</b>
2,<p class='corona19_no'>39998</p>,128347,2021-05-11,강북구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
3,<p class='corona19_no'>39997</p>,128724,2021-05-11,강남구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
4,<p class='corona19_no'>39996</p>,128478,2021-05-11,양천구,-,감염경로 조사중,<b class='status1'>퇴원</b>


In [73]:
df_all.shape

(8800, 7)

# 데이터 전처리

In [74]:
import re
def extract_number(num_string):
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "")
        num = re.sub("[^0-9]", "", num_string)
        num = int(num)
        return num
    else:
        return num_string        

In [75]:
num_string = "<p class='corona19_no'>7625</p>"
extract_number(num_string)

7625

In [76]:
df_all['연번'] = df_all['연번'].map(extract_number)

In [77]:
df_all['연번'].head()

0    40000
1    39999
2    39998
3    39997
4    39996
Name: 연번, dtype: int64

In [78]:
# 한글 전처리 방법 1
def extract_hanguel(origin_text):
    subtract_text = re.sub("[^가-힣]", "", origin_text)
    return subtract_text

In [79]:
extract_hanguel("<b class='status1'>퇴원</b>")

'퇴원'

In [80]:
# 한글 전처리 방법 2
df_all['퇴원현황'].value_counts()

<b class='status1'>퇴원</b>    8755
<b class='status2'>사망</b>      44
<b class='status1'>-</b>        1
Name: 퇴원현황, dtype: int64

In [90]:
df_all.loc[df_all['퇴원현황'].str.contains('퇴원'), "퇴원현황"] = '퇴원'
df_all.loc[df_all['퇴원현황'].str.contains('사망'), "퇴원현황"] = '사망'
df_all.loc[~df_all['퇴원현황'].str.contains('퇴원|사망'), '퇴원현황'] = np.nan
df_all['퇴원현황'].value_counts()

퇴원    8755
사망      44
Name: 퇴원현황, dtype: int64

In [91]:
last_date = df_all.iloc[0]['확진일']
last_date

'2021-05-11'

In [92]:
date = last_date.replace('-', '_')
date

'2021_05_11'

In [93]:
file_name = f"seoul-covid19-{date}.csv"
file_name

'seoul-covid19-2021_05_11.csv'

In [94]:
df_all.to_csv(file_name, index=False)

In [95]:
pd.read_csv(file_name)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,40000,128607,2021-05-11,서초구,-,기타 확진자 접촉,퇴원
1,39999,128576,2021-05-11,양천구,-,양천구 소재 직장 관련('21.5.),퇴원
2,39998,128347,2021-05-11,강북구,-,기타 확진자 접촉,퇴원
3,39997,128724,2021-05-11,강남구,-,기타 확진자 접촉,퇴원
4,39996,128478,2021-05-11,양천구,-,감염경로 조사중,퇴원
...,...,...,...,...,...,...,...
8795,31205,100179,2021-03-24,타시도,-,관악구 직장/인천 집단생활 관련,퇴원
8796,31204,100192,2021-03-24,노원구,-,관악구 직장/인천 집단생활 관련,퇴원
8797,31203,100161,2021-03-24,서대문구,-,감염경로 조사중,퇴원
8798,31202,100165,2021-03-24,강남구,-,감염경로 조사중,퇴원


In [4]:
import pandas as pd
import numpy as np

In [ ]:
# 사이트 변경 이전 데이터 크롤링 방법
# url = "http://seoul.go.kr/coronaV/coronaStatus.do
# table = pd.read_html(url)

# read_html 은 사이트 내의 html 태그를 불러옴
# table[0].T

# table 변수 안에 있는 table 태그의 값을 하나씩 읽어보며 확진자 목록이 있는 테이블을 가져옴
# df = table[4]

# last_day = df.loc[0, "확진일"]
# last_day = last_day.replace(".", "_")

#file_name = f"seoul-covid19_{last_day}.scv"

#df.to_csv(file_name, index=False)
#엑셀로 저장시 encoding='cp949'

#pd.read_csv(file_name)

#### 데이터 수집 방법
1. 수집하고자 하는 페이지의 URL을 알아본다.
2. 파이썬의 작은 브라우저 requests를 통해 URL에 접근한다.
3. response.status_code가 200 OK 라면 정상 응답입니다.
4. request의 response값에서 response.txt만 받아옵니다.
5. html 텍스트를 받아왔다면 bs(reponse.text, 'html.parser')로 html을 해석합니다.
6. soup.select를 통해 원하는 태그에 접근합니다.
7. 목록을 먼저 받아옵니다.
8. 목록에서 행을 하나씩 가져옵니다.
9. 행을 모아서 데이터프레임으로 만듭니다.

#### robot.txt를 확인할 것
disallow: /일 경우 수집 금지

#### chrome 소스코드 보는 법
1. 검사
2. network > XHR or JS or Doc > URL
3. Preview에서 JSON 형식으로 볼 수 있음
ex) data = data_json["data"]
    pd.DataFrame(data)
    
### 서울시 코로나 발생현황 데이터 수집 순서
1. 페이지별 데이터를 수집
2. 전체 페이지를 수집
3. pd.concat([전체데이터리스트])로 떼이터를 하나로 병합
4. 데이터 전처리(html 태그 제거)
5. to_csv로 전체 데이터 병합
6. pd.read_csv로 데이터가 잘 저장되었는지 읽어와서 확인

In [2]:
url = "http://www.seoul.go.kr/coronaV/coronaStatus.do"
url

'http://www.seoul.go.kr/coronaV/coronaStatus.do'

In [5]:
table = pd.read_html(url)

In [6]:
len(table)

8

In [7]:
# read_html은 사이트 내의 html 태그를 불러옴
table[0]

,강남구,강동구,강북구,강서구,관악구,광진구,구로구,금천구,노원구,도봉구,동대문구,동작구,마포구
0,6389,3365,2296,3966,4946,3078,3475,1790,3544,2311,2968,3817,3350
1,+45,+25,+13,+30,+29,+19,+65,+17,+23,+17,+24,+26,+47
2,서대문구,서초구,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구,기타
3,2423,3995,2548,3353,5460,2995,3969,2500,3925,1368,1392,3408,6452
4,+29,+24,+17,+20,+67,+17,+42,+7,+30,+12,+20,+8,+46


In [8]:
# table 변수 안에 있는 table 태그값을 하나씩 읽어보며 확진자 목록이 있는 테이블을 가져옴
table[1]

,강남구,강동구,강북구,강서구,관악구,광진구
0,6389,3365,2296,3966,4946,3078
1,+45,+25,+13,+30,+29,+19
2,구로구,금천구,노원구,도봉구,동대문구,동작구
3,3475,1790,3544,2311,2968,3817
4,+65,+17,+23,+17,+24,+26
5,마포구,서대문구,서초구,성동구,성북구,송파구
6,3350,2423,3995,2548,3353,5460
7,+47,+29,+24,+17,+20,+67
8,양천구,영등포구,용산구,은평구,종로구,중구
9,2995,3969,2500,3925,1368,1392


In [68]:
table[5]

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황


# 11월 중순 이후 데이터 공개방식 변경

In [14]:
import requests

In [19]:
#f-string
url = "https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax_pre.php?draw=1"
# url = f"{url}&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=desc"
url = f"{url}&start=0&length=100"
# url = f"{url}&search%5Bvalue%5D=&search%5Bregex%5D=true&_=1631758944138"
url

'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax_pre.php?draw=1&start=0&length=100'

In [24]:
# 소스코드가 get이므로 get 사용
response = requests.get(url)
data_json = response.json()
pd.DataFrame(data_json["data"])

,0,1,2,3,4,5,6
0,<p class='corona19_no'>40000</p>,128607,2021-05-11,서초구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
1,<p class='corona19_no'>39999</p>,128576,2021-05-11,양천구,-,양천구 소재 직장 관련('21.5.),<b class='status1'>퇴원</b>
2,<p class='corona19_no'>39998</p>,128347,2021-05-11,강북구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
3,<p class='corona19_no'>39997</p>,128724,2021-05-11,강남구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
4,<p class='corona19_no'>39996</p>,128478,2021-05-11,양천구,-,감염경로 조사중,<b class='status1'>퇴원</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>39905</p>,128645,2021-05-11,타시도,-,감염경로 조사중,<b class='status1'>퇴원</b>
96,<p class='corona19_no'>39904</p>,128539,2021-05-11,중랑구,-,타시도 확진자 접촉,<b class='status1'>퇴원</b>
97,<p class='corona19_no'>39903</p>,128508,2021-05-11,도봉구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
98,<p class='corona19_no'>39902</p>,128512,2021-05-11,도봉구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>


In [26]:
records_total = data_json['recordsTotal']
records_total

40000

# 함수로 만들기

In [29]:
def get_seoul_covid19_100(page_no):
    """
    page_no : 입력값으로 페이지 번호를 입력하면 해당 번호의 데이터를 가져옴
    start_no : 입력받은 page_no로
    """
    start_no = (page_no - 1) * 100
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax_pre.php?draw={page_no}"
    url = f"{url}&order%5B0%5D%5Bdir%5D=desc&start={start_no}&length=100"
    response = requests.get(url)
    data_json = response.json()
    return data_json

In [30]:
# 데이터가 있을 때
get_seoul_covid19_100(1)

{'draw': 1,
 'recordsTotal': 40000,
 'recordsFiltered': 40000,
 'data': [["<p class='corona19_no'>40000</p>",
   '128607',
   '2021-05-11',
   '서초구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39999</p>",
   '128576',
   '2021-05-11',
   '양천구',
   '-',
   "양천구 소재 직장 관련('21.5.)",
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39998</p>",
   '128347',
   '2021-05-11',
   '강북구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39997</p>",
   '128724',
   '2021-05-11',
   '강남구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39996</p>",
   '128478',
   '2021-05-11',
   '양천구',
   '-',
   '감염경로 조사중',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39995</p>",
   '128410',
   '2021-05-11',
   '노원구',
   '-',
   '감염경로 조사중',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39994</p>",
   '128572',
   '2021-05-11',
   '금천구',
   '-',
   '감염경로 조사중',
  

In [31]:
get_seoul_covid19_100(2)

{'draw': 2,
 'recordsTotal': 40000,
 'recordsFiltered': 40000,
 'data': [["<p class='corona19_no'>39900</p>",
   '128409',
   '2021-05-11',
   '관악구',
   '-',
   '감염경로 조사중',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39899</p>",
   '128443',
   '2021-05-11',
   '동작구',
   '-',
   '강남구 소재 학원 관련',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39898</p>",
   '128527',
   '2021-05-11',
   '중랑구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39897</p>",
   '128511',
   '2021-05-11',
   '도봉구',
   '-',
   '감염경로 조사중',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39896</p>",
   '128522',
   '2021-05-11',
   '중랑구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39895</p>",
   '128507',
   '2021-05-11',
   '강북구',
   '-',
   '기타 확진자 접촉',
   "<b class='status1'>퇴원</b>"],
  ["<p class='corona19_no'>39894</p>",
   '127848',
   '2021-05-10',
   '기타',
   '-',
   '노원구 직장/지인모임 관련',
   "<

In [32]:
# 데이터가 없을 때
get_seoul_covid19_100(1000)

{'draw': 1000, 'recordsTotal': 40000, 'recordsFiltered': 40000, 'data': []}

# 전체 데이터 가져오기

In [33]:
# time.sleep으로 시차를 두기 위해
import time
# tqdm: 진행상태를 표시하기 위해
from tqdm import trange

In [52]:
# 전체 페이지를 가져오기 전에 일부 페이지만 실행
page_list = []
# 데이터가 제대로 수집 되는지 앞 부분 3페이지 정도만 확인하고 전체 페이지를 수집
# 처음부터 전체 페이지를 수집하면 중간에 오류가 발생해도 찾기 어렵기 때문
all_page = 3
for page_no in trange(1, all_page + 1):
    one_page = get_seoul_covid19_100(page_no)
    if len(one_page["data"]) > 0:
        one_page = pd.DataFrame(one_page["data"])
        page_list.append(one_page)
# 서버에 부담을 주지 않기 위해 0.5초 텀
        time.sleep(0.5)
    else:
        break

100%|██████████| 3/3 [00:02<00:00,  1.20it/s]


In [53]:
# 가져온 데이터가 맞는지 슬라이싱으로 확인
# page_list[:2]
pd.concat(page_list)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>40000</p>,128607,2021-05-11,서초구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
1,<p class='corona19_no'>39999</p>,128576,2021-05-11,양천구,-,양천구 소재 직장 관련('21.5.),<b class='status1'>퇴원</b>
2,<p class='corona19_no'>39998</p>,128347,2021-05-11,강북구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
3,<p class='corona19_no'>39997</p>,128724,2021-05-11,강남구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
4,<p class='corona19_no'>39996</p>,128478,2021-05-11,양천구,-,감염경로 조사중,<b class='status1'>퇴원</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>39705</p>,127625,2021-05-09,강남구,-,강남구 소재 초등학교 관련,<b class='status1'>퇴원</b>
96,<p class='corona19_no'>39704</p>,127626,2021-05-09,강남구,-,강남구 소재 초등학교 관련,<b class='status1'>퇴원</b>
97,<p class='corona19_no'>39703</p>,127689,2021-05-09,광진구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
98,<p class='corona19_no'>39702</p>,127518,2021-05-09,노원구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>


In [58]:
def get_multi_page_list(start_page, end_page):
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        one_page = get_seoul_covid19_100(page_no)
        if len(one_page['data']) > 0:
            one_page = pd.DataFrame(one_page['data'])
            page_list.append(one_page)
            time.sleep(0.5)
        else:
            return page_list
    return page_list

In [61]:
# 데이터 확인
start_page = 1
end_page = 88
page_list = get_multi_page_list(start_page, end_page)
page_list[:1]

100%|██████████| 88/88 [01:39<00:00,  1.13s/it]


[                                   0       1           2    3  4  \
 0   <p class='corona19_no'>40000</p>  128607  2021-05-11  서초구  -   
 1   <p class='corona19_no'>39999</p>  128576  2021-05-11  양천구  -   
 2   <p class='corona19_no'>39998</p>  128347  2021-05-11  강북구  -   
 3   <p class='corona19_no'>39997</p>  128724  2021-05-11  강남구  -   
 4   <p class='corona19_no'>39996</p>  128478  2021-05-11  양천구  -   
 ..                               ...     ...         ...  ... ..   
 95  <p class='corona19_no'>39905</p>  128645  2021-05-11  타시도  -   
 96  <p class='corona19_no'>39904</p>  128539  2021-05-11  중랑구  -   
 97  <p class='corona19_no'>39903</p>  128508  2021-05-11  도봉구  -   
 98  <p class='corona19_no'>39902</p>  128512  2021-05-11  도봉구  -   
 99  <p class='corona19_no'>39901</p>  128331  2021-05-11  용산구  -   
 
                        5                          6  
 0              기타 확진자 접촉  <b class='status1'>퇴원</b>  
 1   양천구 소재 직장 관련('21.5.)  <b class='status1'>퇴원</b>  
 2   

In [63]:
# 하나의 데이터 프레임으로 만듦
df_all = pd.concat(page_list)
df_all.shape

(8800, 7)

In [69]:
cols = table[5].columns.tolist()

In [71]:
df_all.columns = cols

In [72]:
df_all.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,<p class='corona19_no'>40000</p>,128607,2021-05-11,서초구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
1,<p class='corona19_no'>39999</p>,128576,2021-05-11,양천구,-,양천구 소재 직장 관련('21.5.),<b class='status1'>퇴원</b>
2,<p class='corona19_no'>39998</p>,128347,2021-05-11,강북구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
3,<p class='corona19_no'>39997</p>,128724,2021-05-11,강남구,-,기타 확진자 접촉,<b class='status1'>퇴원</b>
4,<p class='corona19_no'>39996</p>,128478,2021-05-11,양천구,-,감염경로 조사중,<b class='status1'>퇴원</b>


In [73]:
df_all.shape

(8800, 7)

# 데이터 전처리

In [74]:
import re
def extract_number(num_string):
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "")
        num = re.sub("[^0-9]", "", num_string)
        num = int(num)
        return num
    else:
        return num_string        

In [75]:
num_string = "<p class='corona19_no'>7625</p>"
extract_number(num_string)

7625

In [76]:
df_all['연번'] = df_all['연번'].map(extract_number)

In [77]:
df_all['연번'].head()

0    40000
1    39999
2    39998
3    39997
4    39996
Name: 연번, dtype: int64

In [78]:
# 한글 전처리 방법 1
def extract_hanguel(origin_text):
    subtract_text = re.sub("[^가-힣]", "", origin_text)
    return subtract_text

In [79]:
extract_hanguel("<b class='status1'>퇴원</b>")

'퇴원'

In [80]:
# 한글 전처리 방법 2
df_all['퇴원현황'].value_counts()

<b class='status1'>퇴원</b>    8755
<b class='status2'>사망</b>      44
<b class='status1'>-</b>        1
Name: 퇴원현황, dtype: int64

In [90]:
df_all.loc[df_all['퇴원현황'].str.contains('퇴원'), "퇴원현황"] = '퇴원'
df_all.loc[df_all['퇴원현황'].str.contains('사망'), "퇴원현황"] = '사망'
df_all.loc[~df_all['퇴원현황'].str.contains('퇴원|사망'), '퇴원현황'] = np.nan
df_all['퇴원현황'].value_counts()

퇴원    8755
사망      44
Name: 퇴원현황, dtype: int64

In [91]:
last_date = df_all.iloc[0]['확진일']
last_date

'2021-05-11'

In [92]:
date = last_date.replace('-', '_')
date

'2021_05_11'

In [93]:
file_name = f"seoul-covid19-{date}.csv"
file_name

'seoul-covid19-2021_05_11.csv'

In [94]:
df_all.to_csv(file_name, index=False)

In [95]:
pd.read_csv(file_name)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,40000,128607,2021-05-11,서초구,-,기타 확진자 접촉,퇴원
1,39999,128576,2021-05-11,양천구,-,양천구 소재 직장 관련('21.5.),퇴원
2,39998,128347,2021-05-11,강북구,-,기타 확진자 접촉,퇴원
3,39997,128724,2021-05-11,강남구,-,기타 확진자 접촉,퇴원
4,39996,128478,2021-05-11,양천구,-,감염경로 조사중,퇴원
...,...,...,...,...,...,...,...
8795,31205,100179,2021-03-24,타시도,-,관악구 직장/인천 집단생활 관련,퇴원
8796,31204,100192,2021-03-24,노원구,-,관악구 직장/인천 집단생활 관련,퇴원
8797,31203,100161,2021-03-24,서대문구,-,감염경로 조사중,퇴원
8798,31202,100165,2021-03-24,강남구,-,감염경로 조사중,퇴원
